In [1]:
from keras import utils

from keras.models import Sequential ,Model
from keras.layers import Input , Dense,Dropout,Flatten , BatchNormalization , Concatenate
from keras.layers import LSTM , Conv1D , ZeroPadding1D , MaxPooling1D ,AveragePooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence,text

from keras import regularizers
from keras.callbacks import EarlyStopping , History

Using TensorFlow backend.


In [4]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pickle
from gensim.models import Word2Vec

para_path='/home/derricksu/model_para/CRNN_2nd'
word2vec_path = os.path.join(para_path , 'word2vec.bin')
print("load model...") 
my_word2vec = Word2Vec.load(word2vec_path)
#print(my_word2vec)

word_freq_path = os.path.join(para_path , 'word_freq.pk')
vocab = dict([(k, v.index) for k, v in my_word2vec.wv.vocab.items()])
weight_matrix = my_word2vec.wv.syn0 #word_to_vec
print("Weight_matrix shape : " , weight_matrix.shape)
del my_word2vec


with open(word_freq_path,"rb") as pk:
    word_freq = pickle.load(pk)
    pk.close()

load model...
Weight_matrix shape :  (53150, 128)


In [6]:
def build(weight_matrix , vocab):
    embedding_vecor_length = weight_matrix.shape[1]
    word_total_index = len(vocab)
    max_review_length = 100
    
    forget_bias_bool=True #default is False
    recu_drop_rate=0.2
    drop_rate=0.2
    back_bool=False #default False
    unroll_bool=True #default is False. This can speed up but consume more memory
    multi_layer_bool=True #default False

    model = Sequential()
    model.add(Embedding(word_total_index, embedding_vecor_length, input_length=max_review_length,
                       weights = [weight_matrix]))
    

    model.add(LSTM(256,
                  unit_forget_bias=forget_bias_bool,
                  recurrent_dropout=recu_drop_rate,
                  dropout=drop_rate,
                  go_backwards=back_bool,
                  unroll=unroll_bool,
                  return_sequences=multi_layer_bool )) 
                   # Control output of this layer , return a decoded sequence whose dimension is same as input
    model.add(Dropout(0.2))

    model.add(LSTM(128,
                  unit_forget_bias=forget_bias_bool,
                  recurrent_dropout=recu_drop_rate,
                  dropout=drop_rate,
                  go_backwards=False,
                  unroll=unroll_bool))
    model.add(Dropout(0.2))

    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))
    
    #model.add(Dense(128,activation='relu'))
    #model.add(Dropout(0.5))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    print(model.summary())

    return model
model=build(weight_matrix , vocab)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          6803200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
__________

In [2]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import LSTM
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length*4)])
X = seq.reshape(-1, length, 2)
y = seq.reshape(-1, length, 2)
# define LSTM configuration
n_neurons = 1#length
n_batch = 1
n_epoch = 1000
# create LSTM
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(length, 2), return_state=True,return_sequences=False))
#model.add(TimeDistributed(Dense(1)))
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary())
# train LSTM
#model.fit(X, y, epochs=n_epoch, batch_size=n_batch, verbose=0)
# evaluate
result = model.predict(X, batch_size=n_batch, verbose=0)
print(result)

ValueError: All layers in a Sequential model should have a single output tensor. For multi-output layers, use the functional API.

In [47]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 2))
lstm1, state_h, state_c = LSTM(1, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3,0.2,0.4,0.6]).reshape((1,3,2))
# make and show prediction
print(model.predict(data))

[array([[-0.10850199]], dtype=float32), array([[-0.10850199]], dtype=float32), array([[-0.27010906]], dtype=float32)]


In [48]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 2))
lstm1, state_h, state_c = LSTM(1, return_sequences=True, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3 , 0.2,0.4,0.6]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

[array([[[ 0.00820675],
        [ 0.01939096],
        [ 0.0311997 ]]], dtype=float32), array([[ 0.0311997]], dtype=float32), array([[ 0.06879383]], dtype=float32)]
